In [22]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Data PreProcessing

### Reading the data

In [23]:
data = pd.read_csv("./dataset/student-mat.csv", sep=';')
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


### Trimming the data

In [24]:
data = data[["studytime", "failures", "freetime", "absences", "G1", "G2", "G3"]]
data.head()

,studytime,failures,freetime,absences,G1,G2,G3
0,2,0,3,6,5,6,6
1,2,0,3,4,5,5,6
2,2,3,3,10,7,8,10
3,3,0,2,2,15,14,15
4,2,0,3,4,6,10,10


In [25]:
featues = data[["studytime", "failures", "freetime", "absences", "G1", "G2"]]
target = data["G3"]

In [26]:
# Scale the data
scaler = StandardScaler()
features_scaled = scaler.fit_transform(featues)

### Split the data

In [27]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=32)

In [28]:
# convert pandas series to numpy
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [29]:
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1,1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1,1)

In [30]:
# checking the dimensions if they match

print(x_test_tensor.shape, x_train_tensor.shape)
print(y_test_tensor.shape, y_train_tensor.shape)

torch.Size([79, 6]) torch.Size([316, 6])
torch.Size([79, 1]) torch.Size([316, 1])


## Creating Regression Model 

In [31]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.layer_1 = nn.Linear(in_features=6, out_features=12)
        self.layer_2 = nn.Linear(in_features=12, out_features=12)
        self.layer_3 = nn.Linear(in_features=12, out_features=1)
        self.relu = nn.ReLU

    def forward(self, x):
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model = RegressionModel()

### Setup Loss Function and Optimizer

In [32]:
loss_fn = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)